# Dev for expressions of quotient derivative

Yuri Shimane, 2021.11.26

In [1]:
import numpy as np
import sympy as sym
import math
from sympy import lambdify

In [2]:
# define parameters
rpmin, k_petro, m_petro, n_petro, r_petro, b_petro = sym.symbols("rpmin k_petro m_petro n_petro r_petro b_petro")
wp = sym.symbols("Wp")
mu = sym.symbols("mu")
f = sym.symbols("f")

In [3]:
# orbital elements
a, e, i, ra, om, ta = sym.symbols("a e i Omega omega theta")

# targeted orbital elements
aT, eT, iT, raT, omT, taT = sym.symbols("a_T e_T i_T Omega_T omega_T theta_T")

# weights on orbital elements
wa, we, wi, wra, wom, wta = sym.symbols("w_a w_e w_i w_Omega w_omega w_theta")

In [4]:
oe = [a,e,i,ra,om,ta]
oe

[a, e, i, Omega, omega, theta]

In [5]:
oeT = [aT, eT, iT, raT, omT, taT]
oeT

[a_T, e_T, i_T, Omega_T, omega_T, theta_T]

In [6]:
woe = [wa, we, wi, wra, wom, wta]
woe

[w_a, w_e, w_i, w_Omega, w_omega, w_theta]

In [7]:
phi1, phi2 = sym.symbols("phi1 phi2")

In [8]:
def angle_difference(phi1, phi2):
    return sym.acos(sym.cos(phi1 - phi2))

In [9]:
expr = sym.sin(i) + sym.cos(i)

In [46]:
foo = lambdify(i, expr)#, 'numpy')

In [47]:
foo(0.29)

1.2441961006175326

In [51]:
doe = [a-aT, e-eT, i-iT, 
       sym.acos(sym.cos(ra - raT)),
       sym.acos(sym.cos(om - omT)),
       sym.acos(sym.cos(ta - taT)),
      ]
doe

[a - a_T,
 e - e_T,
 i - i_T,
 acos(cos(Omega - Omega_T)),
 acos(cos(omega - omega_T)),
 acos(cos(theta - theta_T))]

In [49]:
fun_doe = lambdify([a,e,i,ra,om,ta,aT,eT,iT,raT,omT,taT], doe)  # function version

In [53]:
a_n, e_n, i_n, ra_n, om_n, ta_n = 1.0,0.1,0.2,0.3,0.4,0.5
aT_n, eT_n, iT_n, raT_n, omT_n, taT_n = 1.4,0.6,0.7,0.8,0.9,0.1
fun_doe(a_n, e_n, i_n, ra_n, om_n, ta_n, aT_n, eT_n, iT_n, raT_n, omT_n, taT_n)

[-0.3999999999999999,
 -0.5,
 -0.49999999999999994,
 0.4999999999999999,
 0.4999999999999999,
 0.39999999999999997]

In [190]:
def quotient(mu, f, oe, oeT, rpmin, m_petro, n_petro, r_petro, b_petro, k_petro, wp, woe):
    # unpack elements
    a,e,i,ra,om,ta = oe
    aT, eT, iT, raT, omT, taT = oeT
    rp = a*(1-e)
    p = a*(1 - e**2)
    h = sym.sqrt(a*mu*(1-e**2))
    doe = [a-aT, e-eT, i-iT, 
           sym.acos(sym.cos(ra - raT)),
           sym.acos(sym.cos(om - omT)),
           sym.acos(sym.cos(ta - taT)),
          ]

    # compute RAdot and omdot
    radot = p*f/(h*sym.sin(i)*(sym.sqrt(1 - e**2*sym.cos(om)**2) - e*abs(sym.sin(om))))
    cos_ta_xx_sqrt = sym.sqrt((1/4)*((1-e**2)/e**3)**2 + 1/27)
    ta_xx_term1 = ( (1-e**2) / (2*e**3) + cos_ta_xx_sqrt)**(1/3)
    ta_xx_term2 = (-(1-e**2) / (2*e**3) + cos_ta_xx_sqrt)**(1/3)
    cos_ta_xx = ta_xx_term1 - ta_xx_term2 - 1/e
    r_xx = p/(1+e*cos_ta_xx)
    #inside_sqrt = cos_ta_xx#p**2#*cos_ta_xx**2 #+ (p+r_xx)**2*(1-cos_ta_xx**2)
    omdot_i = f/(e*h)*sym.sqrt(p**2*cos_ta_xx**2 + (p+r_xx)**2*(1-cos_ta_xx**2))
    omdot_o = radot*abs(sym.cos(i))
    omdot = (omdot_i + b_petro*omdot_o)/(1+b_petro)

    # compute oedot
    oedot = [
        2*f*sym.sqrt(a**3 * (1+e)/(mu*(1-e))),
        2*p*f/h,
        p*f/(h*(sym.sqrt(1 - e**2*sym.sin(om)**2) - e*abs(sym.cos(om)))),
        radot,
        omdot,
    ]
    
    # compute periapsis radius constraint P
    p_rp = sym.exp(k_petro*(1.0 - rp/rpmin))
    # compute scaling for each element Soe
    soe = [(1 + ((a-aT)/(m_petro*aT))**n_petro)**(1/r_petro), 1.0, 1.0, 1.0, 1.0, 1.0]
    # compute quotient Q
    sum_term = 0
    for idx in range(5):
        sum_term += woe[idx]*soe[idx]*(doe[idx]/oedot[idx])**2
    q = (1 + wp*p_rp) * sum_term
    fun_q = lambdify(
        [mu, f, oe, oeT, rpmin, m_petro, n_petro, r_petro, b_petro, k_petro, wp, woe],
        q, #omdot_i, 
        "numpy"
    )
    return q, fun_q

In [191]:
q, fun_q = quotient(mu, f, oe, oeT, rpmin, m_petro, n_petro, r_petro, b_petro, k_petro, wp, woe)

In [192]:
a_n, e_n, i_n, ra_n, om_n, ta_n       = 1.0,0.02, 0.2,0.3,0.4,0.5
aT_n, eT_n, iT_n, raT_n, omT_n, taT_n = 1.4,0.1,  0.7,0.8,0.9,0.1

In [193]:
oe_n = [a_n, e_n, i_n, ra_n, om_n, ta_n]
oeT_n = [aT_n, eT_n, iT_n, raT_n, omT_n, taT_n]
mu_n = 1.0
f_n = 1.e-5
rpmin_n = 0.8
m_petro_n = 3.0
n_petro_n = 4.0
r_petro_n = 2.0
b_petro_n = 0.01
k_petro_n = 1.0
wp_n = 1.0
woe_n = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [194]:
fun_q(mu_n, f_n, oe_n, oeT_n, rpmin_n, m_petro_n, n_petro_n, r_petro_n, b_petro_n, k_petro_n, wp_n, woe)

174722908.531766*w_Omega + 691222900.171181*w_a + 28787774.609994*w_e + 4333627746.956*w_i + 458333.031810805*w_omega

In [189]:
-2449.999999691485**(1/3)

-13.480997498313387

In [183]:
e = e_n
0.7937005259841*(math.sqrt(0.148148148148148 + (1 - e)**4/e**6) + (e**2 - 1)/e**3)**0.333333333333333

(6.740498749156692+11.674886301893821j)

In [162]:
(math.sqrt(0.148148148148148 + (1 - e)**4/e**6) + (e**2 - 1)/e**3)#**0.333333333333333

-4899.99999938297

In [164]:
(-4899.99999938297)**(1/3)

(8.492496260852581+14.70943500688538j)

In [148]:
import math

In [61]:
def gauss_var_matrix(mu,oe):
     # unpack elements
    a,e,i,ra,om,ta = oe
    rp = a*(1-e)
    p = a*(1 - e**2)
    h = sym.sqrt(a*mu*(1-e**2))
    r = h**2/(mu*(1+e*sym.cos(ta)))
    # let psi be column major!
    psi = [
        [
            2*a**2/h*e*sym.sin(ta),
            p/h*sym.sin(ta),
            0.0,
            0.0,
            -p/(e*h)*sym.cos(ta)
        ], 
        [
            2*a**2/h * p/r,
            (p+r)/h*sym.cos(ta) + r*e,
            0.0,
            0.0,
            (p+r)/(e*h)*sym.sin(ta)
        ], 
        [
            0.0,
            0.0,
            r*sym.cos(ta+om)/h,
            r*sym.sin(ta+om)/(h*sym.sin(i)),
            -r*sym.sin(ta+om)*sym.cos(i)/(h*sym.sin(i)),
        ]
    ]
    fun_psi = lambdify([mu,oe],psi,"numpy")
    return psi, fun_psi

In [62]:
psi, fun_psi = gauss_var_matrix(mu,oe)

In [20]:
psi[0]

[2*a**2*e*sin(theta)/sqrt(a*mu*(1 - e**2)),
 a*(1 - e**2)*sin(theta)/sqrt(a*mu*(1 - e**2)),
 0.0,
 0.0,
 -a*(1 - e**2)*cos(theta)/(e*sqrt(a*mu*(1 - e**2)))]

In [66]:
fun_psi(mu_n, oe_n)

[[0.09636815917964785, 0.4770223879392568, 0.0, 0.0, -8.731836241047636],
 [2.186476362660801, 1.766933332344911, 0.0, 0.0, 9.155595217418588],
 [0.0, 0.0, 0.5685951871111976, 3.6065954147541763, -3.5347036257960394]]

In [71]:
dqdoe = []
for idx in range(5):
    dqdoe.append(sym.diff(q, oe[idx]))

In [ ]:
ux = -(psi[0][0]*dqdoe[0] + psi[0][1]*dqdoe[1] + psi[0][2]*dqdoe[2] + psi[0][3]*dqdoe[3] + psi[0][4]*dqdoe[4])
uy = -(psi[1][0]*dqdoe[0] + psi[1][1]*dqdoe[1] + psi[1][2]*dqdoe[2] + psi[1][3]*dqdoe[3] + psi[1][4]*dqdoe[4])
uz = -(psi[2][0]*dqdoe[0] + psi[2][1]*dqdoe[1] + psi[2][2]*dqdoe[2] + psi[2][3]*dqdoe[3] + psi[2][4]*dqdoe[4])
fun_ux = lambdify([q,oe], ux)
fun_uy = lambdify([q,oe], uy)
fun_uz = lambdify([q,oe], uz)

In [69]:
def thrust_direction(q, oe):
    dqdoe = []
    for idx in range(5):
        dqdoe.append(sym.diff(q, oe[idx]))
    #psi, _ = gauss_var_matrix(mu,oe)  # to be replaced by generated function
    psi = fun_psi(mu, oe)
    # multiply
    ux = -(psi[0][0]*dqdoe[0] + psi[0][1]*dqdoe[1] + psi[0][2]*dqdoe[2] + psi[0][3]*dqdoe[3] + psi[0][4]*dqdoe[4])
    uy = -(psi[1][0]*dqdoe[0] + psi[1][1]*dqdoe[1] + psi[1][2]*dqdoe[2] + psi[1][3]*dqdoe[3] + psi[1][4]*dqdoe[4])
    uz = -(psi[2][0]*dqdoe[0] + psi[2][1]*dqdoe[1] + psi[2][2]*dqdoe[2] + psi[2][3]*dqdoe[3] + psi[2][4]*dqdoe[4])
    fun_ux = lambdify([q,oe], ux)
    fun_uy = lambdify([q,oe], uy)
    fun_uz = lambdify([q,oe], uz)
    return [ux,uy,uz], [fun_ux, fun_uy, fun_uz]

In [70]:
us, fun_us = thrust_direction(q, oe)
#us = thrust_direction(q, oe)

TypeError: loop of ufunc does not support argument 0 of type Symbol which has no callable sin method

In [31]:
sym.cos(us[1]/us[0])

cos((-2*a**2*(e*cos(theta) + 1)*(-0.0625*Wp*k_petro*mu**4*w_Omega*w_a*w_e*w_i*w_omega*(1 - e)**2*(a - a_T)**2*(b_petro + 1)**2*(e - e_T)**2*(i - i_T)**2*(-e*Abs(sin(omega)) + sqrt(-e**2*cos(omega)**2 + 1))**2*(-e*Abs(cos(omega)) + sqrt(-e**2*sin(omega)**2 + 1))**2*(((a - a_T)/(a_T*m_petro))**n_petro + 1)**(1/r_petro)*exp(k_petro*(-a*(1 - e)/rpmin + 1.0))*sin(i)**2*acos(cos(Omega - Omega_T))**2*acos(cos(omega - omega_T))**2/(a**6*f**8*rpmin*(1 - e**2)**3*(e + 1)*(a*b_petro*f*(1 - e**2)*Abs(cos(i))/(sqrt(a*mu*(1 - e**2))*(-e*Abs(sin(omega)) + sqrt(-e**2*cos(omega)**2 + 1))*sin(i)) + f*sqrt(a**2*(1 - e**2)**2*(0.7937005259841*(sqrt(0.148148148148148 + (1 - e)**4/e**6) + (1 - e**2)/e**3)**0.333333333333333 - 0.7937005259841*(sqrt(0.148148148148148 + (1 - e)**4/e**6) + (e**2 - 1)/e**3)**0.333333333333333 - 1/e)**2 + (1 - (0.7937005259841*(sqrt(0.148148148148148 + (1 - e)**4/e**6) + (1 - e**2)/e**3)**0.333333333333333 - 0.7937005259841*(sqrt(0.148148148148148 + (1 - e)**4/e**6) + (e**2 - 1)/

In [25]:
lambdify([q,oe], us, 'numpy')

SyntaxError: invalid syntax (<lambdifygenerated-6>, line 1)